In [ ]:
#Import the required libraries
import os
import random
import warnings
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import ensemble,neural_network,neighbors,svm,model_selection,inspection #neighbors,svm if we want to use Knearest and SVM models
warnings.simplefilter(action='ignore')

In [ ]:
#Loading the dataset
df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Cleaned2.csv')
df

In [ ]:
# First use all featues to train and validate the models (ANN and RF),in order to detect the important features and best estimator
xFeatures = ['B030','Cu030','K030','N030','Na030','P030','Ptot030','altop','albottom','bdr','ctottop','ctotbottom','catop','cabottom',
            'claytotpsatop','claytotpsabottom','dbodtop','dbodbottom','ececftop','ececfbottom','fetop','febottom','ktop','kbottom','mgtop',
            'mgbottom','ntotncstop','ntotncsbottom','octop','ocbottom','ptop','pbottom','phh2otop','phh2obottom','stop','sbottom','sandtotpsatop',
            'sandtotpsabottom','silttotpsatop','silttotpsabottom','wpg2top','wpg2bottom','zntop','znbottom','SOMtop','SOMbottom','PWPtop',
            'PWPbottom','FCtop','FCbottom','SWStop','SWSbottom','treat2','DEM','slope','TPI','TRI','tr','di','nrd','tmean','tmin','tmax','Nrate','Prate','Krate']
#Identify the dependent variable
yFeatur = 'yield'

In [ ]:
#Specify the workspace INfile and outfile for training
inFile = 'D:/ICARDA/TT2/TestingByControl_1/Final merged file'
outFolder = 'D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/ANN_ICARDA_Training'

In [ ]:
#Statistics, visualization and hyperparameter functions
def performanceStatistics(x, y):  #Statistics function
  E = y - x                       # Error
  AE = np.abs(E)                  # Absolute Error
  MAE = np.mean(AE)               # Mean Absolute Error
  SE = np.power(E, 2)             # Square Error
  MSE = np.mean(SE)               # Mean Square Error (this method is the best)
  RMSE = np.sqrt(MSE)             # Root Mean Square Error
  RB = ((np.mean(y) - np.mean(x)) / np.mean(x)) * 100
  slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
  R2 = np.power(r_value, 2)       # correlation of determination
  stat = {'R2':round(R2, 2),
          'RB':round(RB, 2),
          'MAE':round(MAE, 2),
          'RMSE':round(RMSE, 2),
          'n':len(E)}
  return stat

def plotOriginalPredicted(Original, Predicted, outFile, label=''): #visualization function
  minX = np.min(Original)
  maxX = np.max(Original)
  minY = np.min(Predicted)
  maxY = np.max(Predicted)
  minXY = np.min(np.array([minX, minY]))
  maxXY = np.min(np.array([maxX, maxY]))
  fs = 26
  fig = plt.figure(figsize=(15, 15))
  plt.scatter(Original, Predicted, color='blue', label=label)
  plt.plot(np.linspace(minXY, maxXY, 50), np.linspace(minXY, maxXY, 50), color='red', linestyle='-', linewidth=1, markersize=5, label='1:1 Line')
  plt.xlim([minXY, maxXY])
  plt.ylim([minXY, maxXY])
  plt.xticks(size = fs)
  plt.yticks(size = fs)
  plt.xlabel('Original yield (t/ha)', fontsize=fs)
  plt.ylabel('Predicted yield (t/ha)', fontsize=fs)
  
  stat = performanceStatistics(Original, Predicted)
  digits = 2
  n = round(stat['n'], digits)
  r2 = round(stat['R2'], digits)
  RB = round(stat['RB'], digits)
  MAE = round(stat['MAE'], digits)
  RMSE = round(stat['RMSE'], digits)
  s = 'n={} \n$R^2$={}\nRB={} (%)\nMAE={} (t/ha)\nRMSE={} (t/ha)'.format(n, r2, RB, MAE, RMSE)
  plt.text(x=(minXY + 1), y=(maxXY - 1), s=s, horizontalalignment='left', verticalalignment='top', color='black', fontsize=fs)
  plt.legend(loc= 9, fontsize=fs)
  plt.savefig(outFile, dpi=300, bbox_inches='tight')
  plt.clf()
  plt.close()

def hyperparameters(n):    #Hyperparameters function
  
  RFR_param_grid = dict(n_estimators = [i for i in range(100, 2050, 50)],
                        max_features = ['auto', 'sqrt', 'log2'],
                        )

  hl = [i for i in range(10, 55, 5)] # hidden layers 
  hl = [25] 
  hn = [(n * 2) + 1] # hidden neurons
  hlhn = []
  for i in hl:
    for j in hn:
      hlhn.append((i, j))

  hls = [(i[0], i[1]) for i in hlhn]
  ANN_param_grid = dict(hidden_layer_sizes = hls , 
                        activation = ['identity', 'logistic', 'tanh', 'relu'], # 'identity', 'logistic', 'tanh', 'relu'
                        solver = ['lbfgs', 'sgd', 'adam'], # 'lbfgs', 'sgd', 'adam'
                        max_iter = [1000000],
                        )
  
  #KNN_param_grid = dict(n_neighbors = [i for i in range(10, 55, 5)],
                        #)
   
  #SVR_param_grid = dict(gamma = [0.0625,0.25,0.5,1,2,4],
                        #C = [10, 20, 30, 40],
                        #)
  
  MLA = {
      # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
      'RFR':[ensemble.RandomForestRegressor(random_state=0, verbose=False), RFR_param_grid],
      # https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html
      'ANN':[neural_network.MLPRegressor(random_state=0, verbose=False), ANN_param_grid],
      # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html
      #'KNN':[neighbors.KNeighborsRegressor(n_jobs=-1), KNN_param_grid],
      # https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
      #'SVR':[svm.SVR(verbose=0), SVR_param_grid],
      }

  return MLA

In [ ]:
#Training and validating the models by using 75 % of the datset for training and 25 % for validation
df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Cleaned2.csv')
Xdf = df[xFeatures]
ydf = df[[yFeatur]]

X = df[xFeatures]
y = df[[yFeatur]]
X = (X - Xdf.mean()) / Xdf.std()
y = (y - ydf.mean()) / ydf.std()
df = pd.concat([X, y], axis = 1)
df = df.dropna()
X = df[xFeatures]
y = df[[yFeatur]]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=0)

n = len(xFeatures)
MLA = hyperparameters(n)
for idMLA, mlaL_mlaA in enumerate(MLA.items()):
  mlaL, mlaA = mlaL_mlaA
  estimator = mlaA[0]
  param_grid = mlaA[1]
  GSCV = model_selection.GridSearchCV(estimator, param_grid, scoring='r2', cv=5, refit=True, verbose=False, n_jobs=-1)
  GSCV.fit(X_train, y_train)
  bestEstimator = GSCV.best_estimator_ 

  OriginalYield_test  = y_test.values.flatten()
  PredictedYield_test = bestEstimator.predict(X_test).flatten()
  OriginalYield_test  = (float(ydf.std()) * OriginalYield_test) + float(ydf.mean())
  PredictedYield_test = (float(ydf.std()) * PredictedYield_test) + float(ydf.mean())
  
  outFile     = os.path.join(outFolder, '{}_plot_DefaultML_{}.png'.format(mlaL, n))
  Stats       = os.path.join(outFolder, '{}_Stats_DefaultML_{}.xlsx'.format(mlaL, n))
  Importances = os.path.join(outFolder, '{}_Importances_DefaultML_{}.xlsx'.format(mlaL, n))

  plotOriginalPredicted(OriginalYield_test, PredictedYield_test, outFile, label='Predicted yield of {}'.format(mlaL))
  df_stats = performanceStatistics(OriginalYield_test, PredictedYield_test)
  print(mlaL, df_stats)

  df_GSCV = pd.DataFrame(GSCV.cv_results_)
  df_GSCV = df_GSCV.sort_values(by='rank_test_score', ascending=True)
  df_GSCV = df_GSCV.head(1)
  for i in list(df_GSCV.columns):
    df_stats[i] = df_GSCV[i].values.tolist()[0]
  df_stats = pd.DataFrame.from_dict(df_stats, orient='index').T
  df_stats.to_excel(Stats, index=False)
  
  pi = inspection.permutation_importance(bestEstimator, X_train, y_train, n_jobs=-1, random_state=0).importances_mean
  pi = [((i / pi.sum()) * 100) for i in pi]
  dfImportances = pd.DataFrame(data=[pi], columns=xFeatures).round(2)
  dfImportances.to_excel(Importances, index=False)


In [ ]:
#Testing the best estimator by Control Treatment using all features 
##OutFolder2 where testing ouputs will be placed
outFolder = 'D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/ANN_ICARDA_Testing'
#Test by b Control 
for i in [[xFeaturesML, 'ML'], [xFeaturesML, 'ML']]:

  df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Cleaned2.csv')
  xFeatures = i[0]
  group     = i[1]
  
  print(group)
  Xdf = df[xFeatures]
  ydf = df[[yFeatur]]

  for col in df.columns:
    if col not in ['texture_class_top','texture_class_bottom','treat']:
      mean = np.mean(df[col].values)
      std  = np.std(df[col].values)
      if std != 0:
        df[col] = df[col].apply(lambda x:(x-mean)/std)
  
  testtreat = ['Control'] # random.sample(list(df.treat.unique()), 1)
  print(testtreat)

 
  test  = df[df['treat'].isin(testtreat)]

 
  X_test  = test[xFeatures]
  y_test  = test[[yFeatur]]
#Test by treat(Control) and best estimator
n = len(xFeatures)
  #MLA = hyperparameters(n)
  #for idMLA, mlaL_mlaA in enumerate(MLA.items()):
    #mlaL, mlaA = mlaL_mlaA
    #estimator = mlaA[0]
    #param_grid = mlaA[1]
    #GSCV = model_selection.GridSearchCV(estimator, param_grid, scoring='r2', cv=5, refit=True, verbose=False, n_jobs=-1)
GSCV.predict(X_test)
bestEstimator = GSCV.best_estimator_ 

OriginalYield_test  = y_test.values.flatten()
PredictedYield_test = bestEstimator.predict(X_test).flatten()
OriginalYield_test  = (float(ydf.std()) * OriginalYield_test) + float(ydf.mean())
PredictedYield_test = (float(ydf.std()) * PredictedYield_test) + float(ydf.mean())

yieldFile   = os.path.join(outFolder, '{}_yield_{}_{}.xlsx'.format(mlaL, group, n))
plot        = os.path.join(outFolder, '{}_plot_{}_{}.png'.format(mlaL, group, n))
Stats       = os.path.join(outFolder, '{}_Stats_{}_{}.xlsx'.format(mlaL, group, n))
Importances = os.path.join(outFolder, '{}_Importances_{}_{}.xlsx'.format(mlaL, group, n))

dfYield = pd.DataFrame({'Original Yield':list(OriginalYield_test), 'Predicted Yield':list(PredictedYield_test)})
dfYield.to_excel(yieldFile, index=False)

plotOriginalPredicted(OriginalYield_test, PredictedYield_test, plot, label='Predicted yield of {}'.format(mlaL))
df_stats = performanceStatistics(OriginalYield_test, PredictedYield_test)
print(mlaL, df_stats)

df_GSCV = pd.DataFrame(GSCV.cv_results_)
df_GSCV = df_GSCV.sort_values(by='rank_test_score', ascending=True)
df_GSCV = df_GSCV.head(1)
for i in list(df_GSCV.columns):
       
    df_stats[i] = df_GSCV[i].values.tolist()[0]
            
    df_stats = pd.DataFrame.from_dict(df_stats, orient='index').T
    df_stats.to_excel(Stats, index=False)
    
    pi = inspection.permutation_importance(bestEstimator, X_test, y_test, n_jobs=-1, random_state=0).importances_mean
    pi = [((i / pi.sum()) * 100) for i in pi]
    dfImportances = pd.DataFrame(data=[pi], columns=xFeatures).round(2)
    dfImportances.to_excel(Importances, index=False)
    


In [ ]:
#We found that the best estimator is ANN, and the accuracy is 'R2': 0.52 with using all features and without feature selection.
# Lets try different feature selection approaches such as Univariate and Chi2 alongwith permutation importance from ANN to explore the best approach acheived the highest accuracy

In [96]:
#1-Univariate approach
#Statistical tests can be used to select those features that have the strongest relationship with the output variable.

#The scikit-learn library provides the SelectKBest class that can be used with a suite of different statistical tests to select a specific number of features.

#The example below uses the chi-squared (chi²) statistical test for non-negative features to select k (k=10) of the best features from the Mobile Price Range Prediction Dataset.
#Load the required libraries 
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
df = pd.read_csv("D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned1.csv")

In [97]:
df

,Unnamed: 0,longitude,latitude,B030,Cu030,K030,Mn030,N030,Na030,P030,...,tr,di,nrd,tmean,tmin,tmax,Nrate,Prate,Krate,yield
0,0,29.327895,-1.583471,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,22,0,13.388072
1,1,29.327895,-1.583471,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,0,0,0,11.999783
2,2,29.327895,-1.583471,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,90,40,60,12.359695
3,3,29.327895,-1.583471,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,0,42,12.090858
4,4,29.327895,-1.583471,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,22,42,12.434738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,622,30.058173,-1.650446,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,90,40,60,34.974074
623,623,30.058173,-1.650446,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,51,22,0,26.481260
624,624,30.058173,-1.650446,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,51,22,42,30.263950
625,625,30.058173,-1.650446,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,0,0,0,18.552497


In [98]:
df = df.apply (pd.to_numeric, errors='coerce')

print (df)

     Unnamed: 0  longitude  latitude    B030   Cu030    K030  Mn030     N030  \
0             0  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
1             1  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
2             2  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
3             3  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
4             4  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
..          ...        ...       ...     ...     ...     ...    ...      ...   
622         622  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
623         623  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
624         624  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
625         625  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
626         626  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   

      Na030     P030  ...          tr  

In [85]:
df[df < 0] = 0
df

,Unnamed: 0,longitude,latitude,B030,Cu030,K030,Mn030,N030,Na030,P030,...,tr,di,nrd,tmean,tmin,tmax,Nrate,Prate,Krate,yield
0,0,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,22,0,13.388072
1,1,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,0,0,0,11.999783
2,2,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,90,40,60,12.359695
3,3,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,0,42,12.090858
4,4,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,22,42,12.434738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,622,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,90,40,60,34.974074
623,623,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,51,22,0,26.481260
624,624,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,51,22,42,30.263950
625,625,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,0,0,0,18.552497


In [99]:
df.isnull().values.any()

True

In [100]:
df = df.dropna()
df = df.reset_index(drop=True)

print (df)

     Unnamed: 0  longitude  latitude    B030   Cu030    K030  Mn030     N030  \
0             0  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
1             1  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
2             2  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
3             3  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
4             4  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
..          ...        ...       ...     ...     ...     ...    ...      ...   
604         622  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
605         623  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
606         624  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
607         625  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
608         626  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   

      Na030     P030  ...          tr  

In [101]:
X = df2.iloc[:,1:70]  #independent variables
y = df2.iloc[:,-1]    #target variable i.e price range
X

,longitude,latitude,B030,Cu030,K030,Mn030,N030,Na030,P030,Ptot030,...,TRI,tr,di,nrd,tmean,tmin,tmax,Nrate,Prate,Krate
0,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,1637.36,...,4.875,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,22,0
1,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,1637.36,...,4.875,540.269121,4.357009,70,289.435178,285.229468,294.165451,0,0,0
2,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,1637.36,...,4.875,540.269121,4.357009,70,289.435178,285.229468,294.165451,90,40,60
3,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,1637.36,...,4.875,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,0,42
4,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,1637.36,...,4.875,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,22,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,621.68,...,19.125,459.224912,3.993260,40,291.744169,287.407044,296.738441,90,40,60
623,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,621.68,...,19.125,459.224912,3.993260,40,291.744169,287.407044,296.738441,51,22,0
624,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,621.68,...,19.125,459.224912,3.993260,40,291.744169,287.407044,296.738441,51,22,42
625,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,621.68,...,19.125,459.224912,3.993260,40,291.744169,287.407044,296.738441,0,0,0


In [104]:
y=y.astype('int')
y

0      13
1      11
2      12
3      12
4      12
       ..
622    34
623    26
624    30
625    18
626    26
Name: yield, Length: 609, dtype: int32

In [105]:
#apply SelectKBest class to extract top 20 best features

BestFeatures = SelectKBest(score_func=chi2, k=20)
fit = BestFeatures.fit(X,y)

In [106]:
df_scores = pd.DataFrame(fit.scores_)
df_columns = pd.DataFrame(X.columns)

In [107]:
f_Scores = pd.concat([df_columns,df_scores],axis=1)               # feature scores
f_Scores.columns = ['Specs','Score']  
f_Scores 

,Specs,Score
0,longitude,0.274520
1,latitude,NaN
2,B030,583.039645
3,Cu030,814.486799
4,K030,7626.404234
...,...,...
64,tmin,0.365165
65,tmax,0.791541
66,Nrate,5740.025312
67,Prate,2033.570665


In [108]:
print(f_Scores.nlargest(20,'Score'))       # print 20 best features in descending order

            Specs         Score
29     ntotncstop  5.966576e+07
30  ntotncsbottom  2.732190e+06
15          catop  6.816014e+04
16       cabottom  6.594185e+04
9         Ptot030  1.771189e+04
8            P030  7.830864e+03
4            K030  7.626404e+03
66          Nrate  5.740025e+03
6            N030  5.319171e+03
28       mgbottom  5.031414e+03
25           ktop  4.000623e+03
27          mgtop  3.700923e+03
26        kbottom  2.567191e+03
68          Krate  2.393556e+03
67          Prate  2.033571e+03
23          fetop  1.275079e+03
56            DEM  1.163146e+03
3           Cu030  8.144868e+02
11       albottom  6.681744e+02
59            TRI  5.972794e+02


In [120]:
#Test ANN by these 20 features explored by Univariate approach and see the accuracy
#Univariate Selected features
xFeatures = ['ntotncstop','ntotncsbottom','catop','cabottom','Ptot030','P030','K030','Nrate','N030','mgbottom','ktop','kbottom','Krate','Prate','fetop','DEM','Cu030','albottom','TRI']
xFeaturesML = ['ntotncstop','ntotncsbottom','catop','cabottom','Ptot030','P030','K030','Nrate','N030','mgbottom','ktop','kbottom','Krate','Prate','fetop','DEM','Cu030','albottom','TRI']
yFeatur = 'yield'

In [121]:
#Import the required libraries
import os
import random
import warnings
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import ensemble,neural_network,neighbors,svm,model_selection,inspection #neighbors,svm if we want to use Knearest and SVM models
warnings.simplefilter(action='ignore')

In [122]:
#Training and validating 
df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned3.csv')

Xdf = df[xFeatures]
ydf = df[[yFeatur]]

X = df[xFeatures]
y = df[[yFeatur]]
X = (X - Xdf.mean()) / Xdf.std()
y = (y - ydf.mean()) / ydf.std()
df = pd.concat([X, y], axis = 1)
df = df.dropna()
X = df[xFeatures]
y = df[[yFeatur]]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=0)

n = len(xFeatures)
MLA = hyperparameters(n)
for idMLA, mlaL_mlaA in enumerate(MLA.items()):
  mlaL, mlaA = mlaL_mlaA
  estimator = mlaA[0]
  param_grid = mlaA[1]
  GSCV = model_selection.GridSearchCV(estimator, param_grid, scoring='r2', cv=5, refit=True, verbose=False, n_jobs=-1)
  GSCV.fit(X_train, y_train)
  bestEstimator = GSCV.best_estimator_ 

  OriginalYield_test  = y_test.values.flatten()
  PredictedYield_test = bestEstimator.predict(X_test).flatten()
  OriginalYield_test  = (float(ydf.std()) * OriginalYield_test) + float(ydf.mean())
  PredictedYield_test = (float(ydf.std()) * PredictedYield_test) + float(ydf.mean())
  
  outFile     = os.path.join(outFolder, '{}_plot_DefaultML_{}.png'.format(mlaL, n))
  Stats       = os.path.join(outFolder, '{}_Stats_DefaultML_{}.xlsx'.format(mlaL, n))
  Importances = os.path.join(outFolder, '{}_Importances_DefaultML_{}.xlsx'.format(mlaL, n))

  plotOriginalPredicted(OriginalYield_test, PredictedYield_test, outFile, label='Predicted yield of {}'.format(mlaL))
  df_stats = performanceStatistics(OriginalYield_test, PredictedYield_test)
  print(mlaL, df_stats)

  df_GSCV = pd.DataFrame(GSCV.cv_results_)
  df_GSCV = df_GSCV.sort_values(by='rank_test_score', ascending=True)
  df_GSCV = df_GSCV.head(1)
  for i in list(df_GSCV.columns):
    df_stats[i] = df_GSCV[i].values.tolist()[0]
  df_stats = pd.DataFrame.from_dict(df_stats, orient='index').T
  df_stats.to_excel(Stats, index=False)
  
  pi = inspection.permutation_importance(bestEstimator, X_train, y_train, n_jobs=-1, random_state=0).importances_mean
  pi = [((i / pi.sum()) * 100) for i in pi]
  dfImportances = pd.DataFrame(data=[pi], columns=xFeatures).round(2)
  dfImportances.to_excel(Importances, index=False)

RFR {'R2': 0.78, 'RB': 0.76, 'MAE': 2.05, 'RMSE': 2.96, 'n': 153}
ANN {'R2': 0.8, 'RB': 0.03, 'MAE': 1.98, 'RMSE': 2.77, 'n': 153}


In [126]:
df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned3.csv')
df = df.dropna()
df = df.reset_index(drop=True)

print (df)

     Unnamed: 0  longitude  latitude    B030   Cu030    K030  Mn030     N030  \
0             0  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
1             1  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
2             2  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
3             3  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
4             4  29.327895 -1.583471  108.72  234.68  516.56  58.08  3571.48   
..          ...        ...       ...     ...     ...     ...    ...      ...   
604         622  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
605         623  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
606         624  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
607         625  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   
608         626  30.058173 -1.650446   68.60  147.92  166.16  55.64  3377.08   

      Na030     P030  ...          tr  

In [127]:
#Test by b Control 
for i in [[xFeaturesML, 'ML'], [xFeaturesML, 'ML']]:

  #df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned3.csv')
  xFeatures = i[0]
  group     = i[1]
  
  print(group)
  Xdf = df[xFeatures]
  ydf = df[[yFeatur]]

  for col in df.columns:
    if col not in ['texture_class_top','texture_class_bottom','treat']:
      mean = np.mean(df[col].values)
      std  = np.std(df[col].values)
      if std != 0:
        df[col] = df[col].apply(lambda x:(x-mean)/std)
  
  testtreat = ['Control'] # random.sample(list(df.treat.unique()), 1)
  print(testtreat)

 
  test  = df[df['treat'].isin(testtreat)]

 
  X_test  = test[xFeatures]
  y_test  = test[[yFeatur]]

ML
['Control']
ML
['Control']


In [128]:
###OutFolder2 for test by Control
outFolder = 'D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/UnivariateFS_Test'
#Test by b Control 
for i in [[xFeaturesML, 'ML'], [xFeaturesML, 'ML']]:

  #df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned3.csv')

  xFeatures = i[0]
  group     = i[1]
  
  print(group)
  Xdf = df[xFeatures]
  ydf = df[[yFeatur]]

  for col in df.columns:
    if col not in ['texture_class_top','texture_class_bottom','treat']:
      mean = np.mean(df[col].values)
      std  = np.std(df[col].values)
      if std != 0:
        df[col] = df[col].apply(lambda x:(x-mean)/std)
  
  testtreat = ['Control'] # random.sample(list(df.treat.unique()), 1)
  print(testtreat)

 
  test  = df[df['treat'].isin(testtreat)]

 
  X_test  = test[xFeatures]
  y_test  = test[[yFeatur]]
#Test by treat(Control) and best estimator
n = len(xFeatures)
  #MLA = hyperparameters(n)
  #for idMLA, mlaL_mlaA in enumerate(MLA.items()):
    #mlaL, mlaA = mlaL_mlaA
    #estimator = mlaA[0]
    #param_grid = mlaA[1]
    #GSCV = model_selection.GridSearchCV(estimator, param_grid, scoring='r2', cv=5, refit=True, verbose=False, n_jobs=-1)
GSCV.predict(X_test)
bestEstimator = GSCV.best_estimator_ 

OriginalYield_test  = y_test.values.flatten()
PredictedYield_test = bestEstimator.predict(X_test).flatten()
OriginalYield_test  = (float(ydf.std()) * OriginalYield_test) + float(ydf.mean())
PredictedYield_test = (float(ydf.std()) * PredictedYield_test) + float(ydf.mean())

yieldFile   = os.path.join(outFolder, '{}_yield_{}_{}.xlsx'.format(mlaL, group, n))
plot        = os.path.join(outFolder, '{}_plot_{}_{}.png'.format(mlaL, group, n))
Stats       = os.path.join(outFolder, '{}_Stats_{}_{}.xlsx'.format(mlaL, group, n))
Importances = os.path.join(outFolder, '{}_Importances_{}_{}.xlsx'.format(mlaL, group, n))

dfYield = pd.DataFrame({'Original Yield':list(OriginalYield_test), 'Predicted Yield':list(PredictedYield_test)})
dfYield.to_excel(yieldFile, index=False)

plotOriginalPredicted(OriginalYield_test, PredictedYield_test, plot, label='Predicted yield of {}'.format(mlaL))
df_stats = performanceStatistics(OriginalYield_test, PredictedYield_test)
print(mlaL, df_stats)

df_GSCV = pd.DataFrame(GSCV.cv_results_)
df_GSCV = df_GSCV.sort_values(by='rank_test_score', ascending=True)
df_GSCV = df_GSCV.head(1)
for i in list(df_GSCV.columns):
       
    df_stats[i] = df_GSCV[i].values.tolist()[0]
            
    df_stats = pd.DataFrame.from_dict(df_stats, orient='index').T
    df_stats.to_excel(Stats, index=False)
    
    pi = inspection.permutation_importance(bestEstimator, X_test, y_test, n_jobs=-1, random_state=0).importances_mean
    pi = [((i / pi.sum()) * 100) for i in pi]
    dfImportances = pd.DataFrame(data=[pi], columns=xFeatures).round(2)
    dfImportances.to_excel(Importances, index=False)


ML
['Control']
ML
['Control']
ANN {'R2': 0.72, 'RB': -6.74, 'MAE': 0.25, 'RMSE': 0.31, 'n': 100}


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
#R2 of testing ANN using Univariate approach = 0.72, lets try Chi2 approach

In [ ]:
#2- Chi-square Test

#The Chi-square test is used for categorical features in a dataset. We calculate Chi-square between each feature and the target and select the desired number of features with the best Chi-square scores. In order to correctly apply the chi-squared to test the relation between various features in the dataset and the target variable, the following conditions have to be met: the variables have to be categorical, sampled independently, and values should have an expected frequency greater than 5.

In [133]:
df = pd.read_csv("D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned1.csv")
from sklearn.feature_selection import SelectKBest, chi2

In [135]:
#remove minus values from dataframe
df[df < 0] = 0
df

,Unnamed: 0,longitude,latitude,B030,Cu030,K030,Mn030,N030,Na030,P030,...,tr,di,nrd,tmean,tmin,tmax,Nrate,Prate,Krate,yield
0,0,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,22,0,13.388072
1,1,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,0,0,0,11.999783
2,2,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,90,40,60,12.359695
3,3,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,0,42,12.090858
4,4,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,...,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,22,42,12.434738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,622,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,90,40,60,34.974074
605,623,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,51,22,0,26.481260
606,624,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,51,22,42,30.263950
607,625,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,...,459.224912,3.993260,40,291.744169,287.407044,296.738441,0,0,0,18.552497


In [136]:
df = df.dropna()
df = df.reset_index(drop=True)

print (df)

     Unnamed: 0  longitude  latitude    B030   Cu030    K030  Mn030     N030  \
0             0  29.327895       0.0  108.72  234.68  516.56  58.08  3571.48   
1             1  29.327895       0.0  108.72  234.68  516.56  58.08  3571.48   
2             2  29.327895       0.0  108.72  234.68  516.56  58.08  3571.48   
3             3  29.327895       0.0  108.72  234.68  516.56  58.08  3571.48   
4             4  29.327895       0.0  108.72  234.68  516.56  58.08  3571.48   
..          ...        ...       ...     ...     ...     ...    ...      ...   
604         622  30.058173       0.0   68.60  147.92  166.16  55.64  3377.08   
605         623  30.058173       0.0   68.60  147.92  166.16  55.64  3377.08   
606         624  30.058173       0.0   68.60  147.92  166.16  55.64  3377.08   
607         625  30.058173       0.0   68.60  147.92  166.16  55.64  3377.08   
608         626  30.058173       0.0   68.60  147.92  166.16  55.64  3377.08   

      Na030     P030  ...          tr  

In [140]:
X = df.iloc[:,1:70]  #independent variables
y = df.iloc[:,-1]    #target variable i.e price range
X

,longitude,latitude,B030,Cu030,K030,Mn030,N030,Na030,P030,Ptot030,...,TRI,tr,di,nrd,tmean,tmin,tmax,Nrate,Prate,Krate
0,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,1637.36,...,4.875,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,22,0
1,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,1637.36,...,4.875,540.269121,4.357009,70,289.435178,285.229468,294.165451,0,0,0
2,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,1637.36,...,4.875,540.269121,4.357009,70,289.435178,285.229468,294.165451,90,40,60
3,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,1637.36,...,4.875,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,0,42
4,29.327895,0.0,108.72,234.68,516.56,58.08,3571.48,175.80,1801.80,1637.36,...,4.875,540.269121,4.357009,70,289.435178,285.229468,294.165451,51,22,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
604,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,621.68,...,19.125,459.224912,3.993260,40,291.744169,287.407044,296.738441,90,40,60
605,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,621.68,...,19.125,459.224912,3.993260,40,291.744169,287.407044,296.738441,51,22,0
606,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,621.68,...,19.125,459.224912,3.993260,40,291.744169,287.407044,296.738441,51,22,42
607,30.058173,0.0,68.60,147.92,166.16,55.64,3377.08,157.96,1432.92,621.68,...,19.125,459.224912,3.993260,40,291.744169,287.407044,296.738441,0,0,0


In [142]:
y=y.astype('int')
y

0      13
1      11
2      12
3      12
4      12
       ..
604    34
605    26
606    30
607    18
608    26
Name: yield, Length: 609, dtype: int32

In [143]:
#convert categorical data to int
X_cat = X.astype(int)
#20 features of the high ch2 will be selected
chi2_features = SelectKBest(chi2,k=20)
X_kbest_features = chi2_features.fit_transform(X_cat, y)

#Reduced features
print('Original feature number:',X_cat.shape[1])
print('Reduced feature number:',X_kbest_features.shape[1])

Original feature number: 69
Reduced feature number: 20


In [144]:
print(X_kbest_features.shape)
print(X_kbest_features)

(609, 20)
[[ 234  516 3571 ...   51   22    0]
 [ 234  516 3571 ...    0    0    0]
 [ 234  516 3571 ...   90   40   60]
 ...
 [ 147  166 3377 ...   51   22   42]
 [ 147  166 3377 ...    0    0    0]
 [ 147  166 3377 ...   51    0   42]]


In [145]:
X_kbest_features.astype('float')

array([[ 234.,  516., 3571., ...,   51.,   22.,    0.],
       [ 234.,  516., 3571., ...,    0.,    0.,    0.],
       [ 234.,  516., 3571., ...,   90.,   40.,   60.],
       ...,
       [ 147.,  166., 3377., ...,   51.,   22.,   42.],
       [ 147.,  166., 3377., ...,    0.,    0.,    0.],
       [ 147.,  166., 3377., ...,   51.,    0.,   42.]])

In [146]:
#Array to list to enalble us reading the feature values
arr = np.array(X_kbest_features)
list = arr.tolist()
print(f'List: {list}')

List: [[234, 516, 3571, 1801, 1637, 283, 1872, 1555, 91, 277, 187, 256, 245, 235735, 27041, 2003, 4, 51, 22, 0], [234, 516, 3571, 1801, 1637, 283, 1872, 1555, 91, 277, 187, 256, 245, 235735, 27041, 2003, 4, 0, 0, 0], [234, 516, 3571, 1801, 1637, 283, 1872, 1555, 91, 277, 187, 256, 245, 235735, 27041, 2003, 4, 90, 40, 60], [234, 516, 3571, 1801, 1637, 283, 1872, 1555, 91, 277, 187, 256, 245, 235735, 27041, 2003, 4, 51, 0, 42], [234, 516, 3571, 1801, 1637, 283, 1872, 1555, 91, 277, 187, 256, 245, 235735, 27041, 2003, 4, 51, 22, 42], [234, 516, 3571, 1801, 1637, 283, 1872, 1555, 91, 277, 187, 256, 245, 235735, 27041, 2003, 4, 0, 22, 42], [234, 516, 3571, 1801, 1637, 283, 1872, 1555, 91, 277, 187, 256, 245, 235735, 27041, 2003, 4, 51, 0, 42], [234, 516, 3571, 1801, 1637, 283, 1872, 1555, 91, 277, 187, 256, 245, 235735, 27041, 2003, 4, 51, 22, 0], [234, 516, 3571, 1801, 1637, 283, 1872, 1555, 91, 277, 187, 256, 245, 235735, 27041, 2003, 4, 90, 40, 60], [234, 516, 3571, 1801, 1637, 283, 1872

In [ ]:
#The selected 20 features from Chi2 approach are ['Cu030','K030','P030','Ptot030','albottom','catop','cabottom','fetop','ktop','kbottom','mgtop','mgbottom','ntotncstop','ntotncsbottom','DEM','TRI','Nrate','Prate','Krate']
#Using the same way used above with Univariate, it was found that the accuracy from Chi2 in testing Control= ANN {'R2': 0.65, 'RB': 4.75, 'MAE': 1.79, 'RMSE': 2.27, 'n': 100}. It looks like that there is an opportunity for improving feature selection. Lets try feature permutation importance and compare its accuracy with Univariate and Chi2 approaches.

In [1]:
#Train, validate and test the best estimator and detect the important features through permutation in ANN 
#Load the required libraries
import os
import random
import warnings
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import ensemble,neural_network,neighbors,svm,model_selection,inspection
warnings.simplefilter(action='ignore')

In [2]:
df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned3.csv')


In [3]:
#Specify X and yfeatures
xFeatures = ['B030','Cu030','K030','Mn030','N030','Na030','P030','Ptot030','altop','albottom','bdr','ctottop',
'ctotbottom','catop','cabottom','claytotpsatop','claytotpsabottom','dbodtop','dbodbottom','ececftop',
'ececfbottom','fetop','febottom','ktop','kbottom','mgtop','mgbottom','ntotncstop','ntotncsbottom',
'octop','ocbottom','ptop','pbottom','phh2otop','phh2obottom','stop','sbottom','sandtotpsatop','sandtotpsabottom',
'silttotpsatop','silttotpsabottom','wpg2top','wpg2bottom','zntop','znbottom',
'SOMtop','SOMbottom','PWPtop','PWPbottom','FCtop','FCbottom','SWStop','SWSbottom','treat2','DEM','slope',
'TPI','TRI','tr','di','nrd','tmean','tmin','tmax','Nrate','Prate','Krate']


xFeaturesML = ['B030','Cu030','K030','Mn030','N030','Na030','P030','Ptot030','altop','albottom','bdr','ctottop',
'ctotbottom','catop','cabottom','claytotpsatop','claytotpsabottom','dbodtop','dbodbottom','ececftop',
'ececfbottom','fetop','febottom','ktop','kbottom','mgtop','mgbottom','ntotncstop','ntotncsbottom',
'octop','ocbottom','ptop','pbottom','phh2otop','phh2obottom','stop','sbottom','sandtotpsatop','sandtotpsabottom',
'silttotpsatop','silttotpsabottom','wpg2top','wpg2bottom','zntop','znbottom',
'SOMtop','SOMbottom','PWPtop','PWPbottom','FCtop','FCbottom','SWStop','SWSbottom','treat2','DEM','slope',
'TPI','TRI','tr','di','nrd','tmean','tmin','tmax','Nrate','Prate','Krate']


#xFeaturesHM = ['B030','Cu030','K030','N030','Na030','P030','Ptot030','altop','albottom','bdr','ctottop','ctotbottom','catop','cabottom',
            #'claytotpsatop','claytotpsabottom','dbodtop','dbodbottom','ececftop','ececfbottom','fetop','febottom','ktop','kbottom','mgtop',
            #'mgbottom','ntotncstop','ntotncsbottom','octop','ocbottom','ptop','pbottom','phh2otop','phh2obottom','stop','sbottom','sandtotpsatop',
            #'sandtotpsabottom','silttotpsatop','silttotpsabottom','wpg2top','wpg2bottom','zntop','znbottom','SOMtop','SOMbottom','PWPtop',
            #'PWPbottom','FCtop','FCbottom','SWStop','SWSbottom','treat2','DEM','slope','TPI','TRI','tr','di','nrd','tmean','tmin','tmax','Nrate','Prate','Krate']

yFeatur = 'yield'

In [4]:
#Workspace
inFile = 'D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE'
outFolder = 'D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/ANNFTraining'

In [5]:
#Required functions
#Statistics, visualization and hyperparameter functions
#Statistics, visualization and hyperparameter functions
def performanceStatistics(x, y):  #Statistics function
  E = y - x                       # Error
  AE = np.abs(E)                  # Absolute Error
  MAE = np.mean(AE)               # Mean Absolute Error
  SE = np.power(E, 2)             # Square Error
  MSE = np.mean(SE)               # Mean Square Error (this method is the best)
  RMSE = np.sqrt(MSE)             # Root Mean Square Error
  RB = ((np.mean(y) - np.mean(x)) / np.mean(x)) * 100
  slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
  R2 = np.power(r_value, 2)       # correlation of determination
  stat = {'R2':round(R2, 2),
          'RB':round(RB, 2),
          'MAE':round(MAE, 2),
          'RMSE':round(RMSE, 2),
          'n':len(E)}
  return stat

def plotOriginalPredicted(Original, Predicted, outFile, label=''): #visualization function
  minX = np.min(Original)
  maxX = np.max(Original)
  minY = np.min(Predicted)
  maxY = np.max(Predicted)
  minXY = np.min(np.array([minX, minY]))
  maxXY = np.min(np.array([maxX, maxY]))
  fs = 26
  fig = plt.figure(figsize=(15, 15))
  plt.scatter(Original, Predicted, color='blue', label=label)
  plt.plot(np.linspace(minXY, maxXY, 50), np.linspace(minXY, maxXY, 50), color='red', linestyle='-', linewidth=1, markersize=5, label='1:1 Line')
  plt.xlim([minXY, maxXY])
  plt.ylim([minXY, maxXY])
  plt.xticks(size = fs)
  plt.yticks(size = fs)
  plt.xlabel('Original yield (t/ha)', fontsize=fs)
  plt.ylabel('Predicted yield (t/ha)', fontsize=fs)
  
  stat = performanceStatistics(Original, Predicted)
  digits = 2
  n = round(stat['n'], digits)
  r2 = round(stat['R2'], digits)
  RB = round(stat['RB'], digits)
  MAE = round(stat['MAE'], digits)
  RMSE = round(stat['RMSE'], digits)
  s = 'n={} \n$R^2$={}\nRB={} (%)\nMAE={} (t/ha)\nRMSE={} (t/ha)'.format(n, r2, RB, MAE, RMSE)
  plt.text(x=(minXY + 1), y=(maxXY - 1), s=s, horizontalalignment='left', verticalalignment='top', color='black', fontsize=fs)
  plt.legend(loc= 9, fontsize=fs)
  plt.savefig(outFile, dpi=300, bbox_inches='tight')
  plt.clf()
  plt.close()

def hyperparameters(n):    #Hyperparameters function
  
  RFR_param_grid = dict(n_estimators = [i for i in range(100, 2050, 50)],
                        max_features = ['auto', 'sqrt', 'log2'],
                        )

  hl = [i for i in range(10, 55, 5)] # hidden layers 
  hl = [25] 
  hn = [(n * 2) + 1] # hidden neurons
  hlhn = []
  for i in hl:
    for j in hn:
      hlhn.append((i, j))

  hls = [(i[0], i[1]) for i in hlhn]
  ANN_param_grid = dict(hidden_layer_sizes = hls , 
                        activation = ['identity', 'logistic', 'tanh', 'relu'], # 'identity', 'logistic', 'tanh', 'relu'
                        solver = ['lbfgs', 'sgd', 'adam'], # 'lbfgs', 'sgd', 'adam'
                        max_iter = [1000000],
                        )
  
  #KNN_param_grid = dict(n_neighbors = [i for i in range(10, 55, 5)],
                        #)
   
  #SVR_param_grid = dict(gamma = [0.0625,0.25,0.5,1,2,4],
                        #C = [10, 20, 30, 40],
                        #)
  
  MLA = {
      # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
      'RFR':[ensemble.RandomForestRegressor(random_state=0, verbose=False), RFR_param_grid],
      # https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html
      'ANN':[neural_network.MLPRegressor(random_state=0, verbose=False), ANN_param_grid],
      # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html
      #'KNN':[neighbors.KNeighborsRegressor(n_jobs=-1), KNN_param_grid],
      # https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html
      #'SVR':[svm.SVR(verbose=0), SVR_param_grid],
      }

  return MLA

In [6]:
#Training and validating 
df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned3.csv')
df = df.dropna()
df = df.reset_index(drop=True)

Xdf = df[xFeatures]
ydf = df[[yFeatur]]

X = df[xFeatures]
y = df[[yFeatur]]
X = (X - Xdf.mean()) / Xdf.std()
y = (y - ydf.mean()) / ydf.std()
df = pd.concat([X, y], axis = 1)
df = df.dropna()
X = df[xFeatures]
y = df[[yFeatur]]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=0)

n = len(xFeatures)
MLA = hyperparameters(n)
for idMLA, mlaL_mlaA in enumerate(MLA.items()):
  mlaL, mlaA = mlaL_mlaA
  estimator = mlaA[0]
  param_grid = mlaA[1]
  GSCV = model_selection.GridSearchCV(estimator, param_grid, scoring='r2', cv=5, refit=True, verbose=False, n_jobs=-1)
  GSCV.fit(X_train, y_train)
  bestEstimator = GSCV.best_estimator_ 

  OriginalYield_test  = y_test.values.flatten()
  PredictedYield_test = bestEstimator.predict(X_test).flatten()
  OriginalYield_test  = (float(ydf.std()) * OriginalYield_test) + float(ydf.mean())
  PredictedYield_test = (float(ydf.std()) * PredictedYield_test) + float(ydf.mean())
  
  outFile     = os.path.join(outFolder, '{}_plot_DefaultML_{}.png'.format(mlaL, n))
  Stats       = os.path.join(outFolder, '{}_Stats_DefaultML_{}.xlsx'.format(mlaL, n))
  Importances = os.path.join(outFolder, '{}_Importances_DefaultML_{}.xlsx'.format(mlaL, n))

  plotOriginalPredicted(OriginalYield_test, PredictedYield_test, outFile, label='Predicted yield of {}'.format(mlaL))
  df_stats = performanceStatistics(OriginalYield_test, PredictedYield_test)
  print(mlaL, df_stats)

  df_GSCV = pd.DataFrame(GSCV.cv_results_)
  df_GSCV = df_GSCV.sort_values(by='rank_test_score', ascending=True)
  df_GSCV = df_GSCV.head(1)
  for i in list(df_GSCV.columns):
       
    df_stats[i] = df_GSCV[i].values.tolist()[0]
  df_stats = pd.DataFrame.from_dict(df_stats, orient='index').T
  df_stats.to_excel(Stats, index=False)
  
  pi = inspection.permutation_importance(bestEstimator, X_train, y_train, n_jobs=-1, random_state=0).importances_mean
  pi = [((i / pi.sum()) * 100) for i in pi]
  dfImportances = pd.DataFrame(data=[pi], columns=xFeatures).round(2)
  dfImportances.to_excel(Importances, index=False)

RFR {'R2': 0.85, 'RB': 1.06, 'MAE': 1.69, 'RMSE': 2.46, 'n': 153}
ANN {'R2': 0.78, 'RB': 1.0, 'MAE': 2.24, 'RMSE': 2.92, 'n': 153}


In [7]:
##OutFolder2 for testing outputs
outFolder = 'D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/ANNFTesting'

In [8]:
#Test by b Control 
for i in [[xFeaturesML, 'ML'], [xFeaturesML, 'ML']]:

  df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned3.csv')
  df = df.dropna()
  df = df.reset_index(drop=True)

  xFeatures = i[0]
  group     = i[1]
  
  print(group)
  Xdf = df[xFeatures]
  ydf = df[[yFeatur]]

  for col in df.columns:
    if col not in ['texture_class_top','texture_class_bottom','treat']:
      mean = np.mean(df[col].values)
      std  = np.std(df[col].values)
      if std != 0:
        df[col] = df[col].apply(lambda x:(x-mean)/std)
  
  testtreat = ['Control'] # random.sample(list(df.treat.unique()), 1)
  print(testtreat)

 
  test  = df[df['treat'].isin(testtreat)]

 
  X_test  = test[xFeatures]
  y_test  = test[[yFeatur]]
#Test by treat(Control) and best estimator
n = len(xFeatures)
  #MLA = hyperparameters(n)
  #for idMLA, mlaL_mlaA in enumerate(MLA.items()):
    #mlaL, mlaA = mlaL_mlaA
    #estimator = mlaA[0]
    #param_grid = mlaA[1]
    #GSCV = model_selection.GridSearchCV(estimator, param_grid, scoring='r2', cv=5, refit=True, verbose=False, n_jobs=-1)
GSCV.predict(X_test)
bestEstimator = GSCV.best_estimator_ 

OriginalYield_test  = y_test.values.flatten()
PredictedYield_test = bestEstimator.predict(X_test).flatten()
OriginalYield_test  = (float(ydf.std()) * OriginalYield_test) + float(ydf.mean())
PredictedYield_test = (float(ydf.std()) * PredictedYield_test) + float(ydf.mean())

yieldFile   = os.path.join(outFolder, '{}_yield_{}_{}.xlsx'.format(mlaL, group, n))
plot        = os.path.join(outFolder, '{}_plot_{}_{}.png'.format(mlaL, group, n))
Stats       = os.path.join(outFolder, '{}_Stats_{}_{}.xlsx'.format(mlaL, group, n))
Importances = os.path.join(outFolder, '{}_Importances_{}_{}.xlsx'.format(mlaL, group, n))

dfYield = pd.DataFrame({'Original Yield':list(OriginalYield_test), 'Predicted Yield':list(PredictedYield_test)})
dfYield.to_excel(yieldFile, index=False)

plotOriginalPredicted(OriginalYield_test, PredictedYield_test, plot, label='Predicted yield of {}'.format(mlaL))
df_stats = performanceStatistics(OriginalYield_test, PredictedYield_test)
print(mlaL, df_stats)

df_GSCV = pd.DataFrame(GSCV.cv_results_)
df_GSCV = df_GSCV.sort_values(by='rank_test_score', ascending=True)
df_GSCV = df_GSCV.head(1)
for i in list(df_GSCV.columns):
       
    df_stats[i] = df_GSCV[i].values.tolist()[0]
            
    df_stats = pd.DataFrame.from_dict(df_stats, orient='index').T
    df_stats.to_excel(Stats, index=False)
    
    pi = inspection.permutation_importance(bestEstimator, X_test, y_test, n_jobs=-1, random_state=0).importances_mean
    pi = [((i / pi.sum()) * 100) for i in pi]
    dfImportances = pd.DataFrame(data=[pi], columns=xFeatures).round(2)
    dfImportances.to_excel(Importances, index=False)
    

ML
['Control']
ML
['Control']
ANN {'R2': 0.56, 'RB': 1.7, 'MAE': 2.05, 'RMSE': 2.46, 'n': 100}


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
#Accurcy of ANN (the best estimator) using all features and without permutation importance, R2=0.56, lets drop the following features 
#based on the PI, and see the accuracy under selected features ['ntotncsbottom', 'SOMbottom', 'FCbottom', 'octop', 'znbottom', 'ececfbottom', 'mgbottom']

In [ ]:
#ANN important features:59 (the selected features after RFE using permutation importance)

In [28]:
xFeatures = ['B030','Cu030','K030','Mn030','N030','Na030','P030','Ptot030','altop','albottom','bdr','ctottop',
'ctotbottom','cabottom','claytotpsatop','dbodtop','dbodbottom','ececftop',
'fetop','febottom','ktop','kbottom','mgtop','ntotncstop','ocbottom','ptop','pbottom','phh2otop','phh2obottom',
'stop','sbottom','sandtotpsatop','sandtotpsabottom','silttotpsatop','silttotpsabottom','wpg2top','wpg2bottom','zntop',
'PWPtop','PWPbottom','FCtop','SWStop','SWSbottom','treat2','DEM','slope',
'TPI','TRI','tr','di','nrd','tmean','tmin','tmax','Nrate','Prate','Krate'] 

xFeaturesML = ['B030','Cu030','K030','Mn030','N030','Na030','P030','Ptot030','altop','albottom','bdr','ctottop',
'ctotbottom','cabottom','claytotpsatop','dbodtop','dbodbottom','ececftop',
'fetop','febottom','ktop','kbottom','mgtop','ntotncstop','ocbottom','ptop','pbottom','phh2otop','phh2obottom',
'stop','sbottom','sandtotpsatop','sandtotpsabottom','silttotpsatop','silttotpsabottom','wpg2top','wpg2bottom','zntop',
'PWPtop','PWPbottom','FCtop','SWStop','SWSbottom','treat2','DEM','slope',
'TPI','TRI','tr','di','nrd','tmean','tmin','tmax','Nrate','Prate','Krate'] 


#xFeaturesHM = ['B030','Cu030','K030','N030','Na030','P030','Ptot030','altop','albottom','bdr','ctottop','ctotbottom','catop','cabottom',
            #'claytotpsatop','claytotpsabottom','dbodtop','dbodbottom','ececftop','ececfbottom','fetop','febottom','ktop','kbottom','mgtop',
            #'mgbottom','ntotncstop','ntotncsbottom','octop','ocbottom','ptop','pbottom','phh2otop','phh2obottom','stop','sbottom','sandtotpsatop',
            #'sandtotpsabottom','silttotpsatop','silttotpsabottom','wpg2top','wpg2bottom','zntop','znbottom','SOMtop','SOMbottom','PWPtop',
            #'PWPbottom','FCtop','FCbottom','SWStop','SWSbottom','treat2','DEM','slope','TPI','TRI','tr','di','nrd','tmean','tmin','tmax','Nrate','Prate','Krate']

yFeatur = 'yield'

In [29]:
inFile = 'D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE'
outFolder = 'D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/ANNFPITraining'

In [30]:
###Default training 

In [31]:
df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned3.csv')
df = df.dropna()
df = df.reset_index(drop=True)

Xdf = df[xFeatures]
ydf = df[[yFeatur]]

X = df[xFeatures]
y = df[[yFeatur]]
X = (X - Xdf.mean()) / Xdf.std()
y = (y - ydf.mean()) / ydf.std()
df = pd.concat([X, y], axis = 1)
df = df.dropna()
X = df[xFeatures]
y = df[[yFeatur]]
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25, random_state=0)

n = len(xFeatures)
MLA = hyperparameters(n)
for idMLA, mlaL_mlaA in enumerate(MLA.items()):
  mlaL, mlaA = mlaL_mlaA
  estimator = mlaA[0]
  param_grid = mlaA[1]
  GSCV = model_selection.GridSearchCV(estimator, param_grid, scoring='r2', cv=5, refit=True, verbose=False, n_jobs=-1)
  GSCV.fit(X_train, y_train)
  bestEstimator = GSCV.best_estimator_ 

  OriginalYield_test  = y_test.values.flatten()
  PredictedYield_test = bestEstimator.predict(X_test).flatten()
  OriginalYield_test  = (float(ydf.std()) * OriginalYield_test) + float(ydf.mean())
  PredictedYield_test = (float(ydf.std()) * PredictedYield_test) + float(ydf.mean())
  
  outFile     = os.path.join(outFolder, '{}_plot_DefaultML_{}.png'.format(mlaL, n))
  Stats       = os.path.join(outFolder, '{}_Stats_DefaultML_{}.xlsx'.format(mlaL, n))
  Importances = os.path.join(outFolder, '{}_Importances_DefaultML_{}.xlsx'.format(mlaL, n))

  plotOriginalPredicted(OriginalYield_test, PredictedYield_test, outFile, label='Predicted yield of {}'.format(mlaL))
  df_stats = performanceStatistics(OriginalYield_test, PredictedYield_test)
  print(mlaL, df_stats)

  df_GSCV = pd.DataFrame(GSCV.cv_results_)
  df_GSCV = df_GSCV.sort_values(by='rank_test_score', ascending=True)
  df_GSCV = df_GSCV.head(1)
  for i in list(df_GSCV.columns):
    df_stats[i] = df_GSCV[i].values.tolist()[0]
  df_stats = pd.DataFrame.from_dict(df_stats, orient='index').T
  df_stats.to_excel(Stats, index=False)
  
  pi = inspection.permutation_importance(bestEstimator, X_train, y_train, n_jobs=-1, random_state=0).importances_mean
  pi = [((i / pi.sum()) * 100) for i in pi]
  dfImportances = pd.DataFrame(data=[pi], columns=xFeatures).round(2)
  dfImportances.to_excel(Importances, index=False)

RFR {'R2': 0.85, 'RB': 1.07, 'MAE': 1.68, 'RMSE': 2.45, 'n': 153}
ANN {'R2': 0.85, 'RB': -0.87, 'MAE': 1.87, 'RMSE': 2.45, 'n': 153}


In [32]:
##OutFolder2
outFolder = 'D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/ANNFPITesting'

In [33]:
#Test by b Control 
for i in [[xFeaturesML, 'ML'], [xFeaturesML, 'ML']]:

  df = pd.read_csv('D:/ICARDA/TT2/TestingByControl_1/Final merged file/Kheir_Codes and results for building ML to predict potato in Rwanda/Feature selection techniques_basedRFE/Cleaned3.csv')
  df = df.dropna()
  df = df.reset_index(drop=True)

  xFeatures = i[0]
  group     = i[1]
  
  print(group)
  Xdf = df[xFeatures]
  ydf = df[[yFeatur]]

  for col in df.columns:
    if col not in ['texture_class_top','texture_class_bottom','treat']:
      mean = np.mean(df[col].values)
      std  = np.std(df[col].values)
      if std != 0:
        df[col] = df[col].apply(lambda x:(x-mean)/std)
  
  testtreat = ['Control'] # random.sample(list(df.treat.unique()), 1)
  print(testtreat)

 
  test  = df[df['treat'].isin(testtreat)]

 
  X_test  = test[xFeatures]
  y_test  = test[[yFeatur]]
#Test by treat(Control) and best estimator
n = len(xFeatures)
  #MLA = hyperparameters(n)
  #for idMLA, mlaL_mlaA in enumerate(MLA.items()):
    #mlaL, mlaA = mlaL_mlaA
    #estimator = mlaA[0]
    #param_grid = mlaA[1]
    #GSCV = model_selection.GridSearchCV(estimator, param_grid, scoring='r2', cv=5, refit=True, verbose=False, n_jobs=-1)
GSCV.predict(X_test)
bestEstimator = GSCV.best_estimator_ 

OriginalYield_test  = y_test.values.flatten()
PredictedYield_test = bestEstimator.predict(X_test).flatten()
OriginalYield_test  = (float(ydf.std()) * OriginalYield_test) + float(ydf.mean())
PredictedYield_test = (float(ydf.std()) * PredictedYield_test) + float(ydf.mean())

yieldFile   = os.path.join(outFolder, '{}_yield_{}_{}.xlsx'.format(mlaL, group, n))
plot        = os.path.join(outFolder, '{}_plot_{}_{}.png'.format(mlaL, group, n))
Stats       = os.path.join(outFolder, '{}_Stats_{}_{}.xlsx'.format(mlaL, group, n))
Importances = os.path.join(outFolder, '{}_Importances_{}_{}.xlsx'.format(mlaL, group, n))

dfYield = pd.DataFrame({'Original Yield':list(OriginalYield_test), 'Predicted Yield':list(PredictedYield_test)})
dfYield.to_excel(yieldFile, index=False)

plotOriginalPredicted(OriginalYield_test, PredictedYield_test, plot, label='Predicted yield of {}'.format(mlaL))
df_stats = performanceStatistics(OriginalYield_test, PredictedYield_test)
print(mlaL, df_stats)

df_GSCV = pd.DataFrame(GSCV.cv_results_)
df_GSCV = df_GSCV.sort_values(by='rank_test_score', ascending=True)
df_GSCV = df_GSCV.head(1)
for i in list(df_GSCV.columns):
       
    df_stats[i] = df_GSCV[i].values.tolist()[0]
            
    df_stats = pd.DataFrame.from_dict(df_stats, orient='index').T
    df_stats.to_excel(Stats, index=False)
    
    pi = inspection.permutation_importance(bestEstimator, X_test, y_test, n_jobs=-1, random_state=0).importances_mean
    pi = [((i / pi.sum()) * 100) for i in pi]
    dfImportances = pd.DataFrame(data=[pi], columns=xFeatures).round(2)
    dfImportances.to_excel(Importances, index=False)

ML
['Control']
ML
['Control']
ANN {'R2': 0.78, 'RB': -0.67, 'MAE': 1.12, 'RMSE': 1.79, 'n': 100}


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
#Now, Permutation importances based ANN, showed the best accuracy of testing by Control (R2=0.78), compared with Univariate (R2=0.72), and Chi2 (R2=0.65)